Dataset: 2D binary classification using make_classification

Model:
y^​=σ(wTx+b)
σ is the sigmoid

Loss: Binary Cross-Entropy

Optimizer: Manual gradient update using basic PyTorch ops

In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [3]:
# Generate binary classification data
X, y = make_classification(
    n_samples=100,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_repeated=0,
    n_classes=2,
    random_state=1
)

# Create DataFrame and save to CSV
df = pd.DataFrame(X, columns=['f1', 'f2'])
df['label'] = y
df.to_csv('binary_data.csv', index=False)

# Show sample
df.head()


,f1,f2,label
0,1.300227,-0.785654,1
1,1.441844,-0.560086,1
2,-0.847924,-1.366213,0
3,-0.722150,-1.411294,0
4,-1.272215,0.259451,0


In [4]:
# Load from CSV
data = pd.read_csv('binary_data.csv')

# Features and labels
X = data[['f1', 'f2']].values
y = data['label'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)


In [5]:
# Sigmoid activation
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

# Binary Cross Entropy loss
def binary_cross_entropy(pred, target):
    epsilon = 1e-8  # avoid log(0)
    return -torch.mean(target * torch.log(pred + epsilon) + (1 - target) * torch.log(1 - pred + epsilon))


In [6]:
# Initialize weights and bias
torch.manual_seed(0)
W = torch.randn(2, 1, requires_grad=True)  # shape (n_features, 1)
b = torch.randn(1, requires_grad=True)

# Learning rate
lr = 0.1
epochs = 100


In [7]:
for epoch in range(1, epochs + 1):
    # Forward pass
    z = X_train @ W + b      # Linear combination
    y_pred = sigmoid(z)      # Apply sigmoid

    # Compute loss
    loss = binary_cross_entropy(y_pred, y_train)

    # Backward pass (autograd)
    loss.backward()

    # Manually update weights
    with torch.no_grad():
        W -= lr * W.grad
        b -= lr * b.grad

        # Zero gradients
        W.grad.zero_()
        b.grad.zero_()

    # Print every 10 epochs
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")


Epoch 1: Loss = 0.6409
Epoch 10: Loss = 0.5099
Epoch 20: Loss = 0.4106
Epoch 30: Loss = 0.3427
Epoch 40: Loss = 0.2947
Epoch 50: Loss = 0.2597
Epoch 60: Loss = 0.2333
Epoch 70: Loss = 0.2128
Epoch 80: Loss = 0.1965
Epoch 90: Loss = 0.1834
Epoch 100: Loss = 0.1726


In [8]:
# Forward pass on test data
with torch.no_grad():
    test_logits = X_test @ W + b
    test_pred = sigmoid(test_logits)
    predicted_labels = (test_pred >= 0.5).float()

# Accuracy
accuracy = (predicted_labels == y_test).float().mean() * 100
print(f"Accuracy on test set = {accuracy:.2f}%")


Accuracy on test set = 100.00%
